# 🧭 Trip Mallorca — *Tu código, seccionado con títulos y pocas partes*
Separé en **pocas** celdas por funcionalidad principal (sin cambiar tu lógica), para que sea legible y funcional.

## 🧰 Librerías e importaciones

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
import re

In [ ]:
def close_all_known_popups(driver):
    try:
        boton = driver.find_element(By.CSS_SELECTOR, '[aria-label="Cerrar"]')
        boton.click()
    except:
        pass

def safe_get(func):
    try:
        return func()
    except:
        return None


## 💻 Iniciar sesión del navegador

In [ ]:
driver = webdriver.Chrome(service=Service())
driver.get("https://www.tripadvisor.es/Attractions-g187463-Activities-c42-Palma_de_Mallorca_Majorca_Balearic_Islands.html")
time.sleep(2)
close_all_known_popups(driver)

# Aceptar cookies
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    ).click()
    print("Cookies aceptadas.")
except:
    print("Banner de cookies no apareció.")

# Cerrar modal de inicio de sesión
try:
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '[role="dialog"] button[aria-label="Dismiss sign-in info."]'))
    ).click()
except:
    print("No apareció modal de inicio de sesión.")


## 🔎 Scrapping

In [ ]:
experiencias = []
max_paginas = 50

for pagina in range(max_paginas):
    time.sleep(2)
    close_all_known_popups(driver)

    tarjetas = driver.find_elements(By.CSS_SELECTOR, 'section[data-automation="WebPresentation_SingleFlexCardSection"]')
    print(f"📄 Página {pagina+1}: {len(tarjetas)} tarjetas encontradas.")

    if len(tarjetas) == 0:
        print("No se encontraron tarjetas. Fin del scraping.")
        break

    for i, tarjeta in enumerate(tarjetas):
        try:
            ActionChains(driver).move_to_element(tarjeta).perform()
            time.sleep(0.5)

            nombre = safe_get(lambda: tarjeta.find_element(By.CSS_SELECTOR, 'h3').text.strip())
            enlace = safe_get(lambda: tarjeta.find_element(By.CSS_SELECTOR, 'a[target="_blank"]').get_attribute("href"))

            puntuacion = None
            for span in tarjeta.find_elements(By.CSS_SELECTOR, 'span[aria-hidden="true"]'):
                texto = span.text.strip()
                if re.match(r"^\d+,\d$", texto):
                    puntuacion = float(texto.replace(",", "."))
                    break

            precio = safe_get(lambda: tarjeta.find_element(By.XPATH, ".//*[contains(text(),'€')]").text.strip())

            numero_reviews = None
            try:
                for div in tarjeta.find_elements(By.CSS_SELECTOR, 'div.biGQs._P.pZUbB.ZNjnF'):
                    texto = div.text.strip().replace('.', '')
                    if texto.isdigit():
                        numero_reviews = int(texto)
                        break
            except:
                pass

            descripcion = safe_get(lambda: tarjeta.find_element(By.CSS_SELECTOR, 'span.SwTtt').text.strip())

            cancelacion_gratis = False
            try:
                texto_cancel = tarjeta.find_element(By.CSS_SELECTOR, 'span.biGQs._P.pZUbB.egaXP.ZNjnF').text
                if "Cancelación gratuita" in texto_cancel:
                    cancelacion_gratis = True
            except:
                pass

            imagen = safe_get(lambda: tarjeta.find_element(By.CSS_SELECTOR, 'img').get_attribute("src"))

            experiencia = {
                "nombre_experiencia": nombre,
                "enlace_experiencia": enlace,
                "puntuacion_experiencia": puntuacion,
                "precio": precio,
                "numero_reviews": numero_reviews,
                "descripcion": descripcion,
                "cancelacion_gratis": cancelacion_gratis,
                "url_imagen_experiencia": imagen
            }

            experiencias.append(experiencia)
            print(f" #{len(experiencias)}: {nombre}")

        except Exception as e:
            print(f"Error en tarjeta #{i+1}: {e}")

    # Pasar a la siguiente página (método robusto con href como respaldo)
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-smoke-attr="pagination-next-arrow"]'))
        )

        href = boton_siguiente.get_attribute("href")
        if href:
            print("Clic en botón 'Siguiente'")
            driver.get(href)
            time.sleep(3)
        else:
            print("No se encontró href en botón 'Siguiente'.")
            break

    except Exception as e:
        print(f"No se pudo pasar a la siguiente página: {e}")
        break

## 💾 Guardar Archivo

In [ ]:
archivo = "experiencias_tripadvisor_mallorca.csv"
with open(archivo, "w", newline="", encoding="utf-8") as f:
    campos = [
        "nombre_experiencia",
        "enlace_experiencia",
        "puntuacion_experiencia",
        "precio",
        "numero_reviews",
        "descripcion",
        "cancelacion_gratis",
        "url_imagen_experiencia"
    ]
    writer = csv.DictWriter(f, fieldnames=campos)
    writer.writeheader()
    writer.writerows(experiencias)

driver.quit()
print(f"✅ Scraping completado. Total: {len(experiencias)} experiencias guardadas en {archivo}")